# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [9]:
import heapq
from collections import deque
import time
import itertools  # Add this for the counter

n = 2

class State:
    def __init__(self, left, island, right, boat_location):
        self.left = left
        self.island = island
        self.right = right
        self.boat_location = boat_location

    def __eq__(self, other):
        return (self.left == other.left and 
                self.island == other.island and 
                self.right == other.right and 
                self.boat_location == other.boat_location)
    
    def __hash__(self):
        return hash((frozenset(self.left), frozenset(self.island), 
                     frozenset(self.right), self.boat_location))
    
    def __str__(self):
        locations = ["Left", "Island", "Right"]
        return f"Left: {sorted(self.left)}\nIsland: {sorted(self.island)}\nRight: {sorted(self.right)}\nBoat at: {locations[self.boat_location]}"
    
    def is_valid(self):
        for location in [self.left, self.island, self.right]:
            wives = {i for i in location if i < 0}  
            husbands = {i for i in location if i > 0} 
            
            for wife in wives:
                wife_id = abs(wife)
                if wife_id not in husbands and any(h != wife_id for h in husbands):
                    return False  
        
        return True
    
    def is_goal(self):
        return len(self.left) == 0 and len(self.island) == 0 and len(self.right) == 2*n
    
    def get_possible_moves(self):
        moves = []
        
        if self.boat_location == 0:
            source, destinations = self.left, [self.island]
            next_boat_location = 1
        elif self.boat_location == 1:  
            source, destinations = self.island, [self.left, self.right]
            next_boat_locations = [0, 2]
        else:  
            source, destinations = self.right, [self.island]
            next_boat_location = 1
        
        for p1 in source:
            for i, dest in enumerate(destinations):
                new_left = set(self.left)
                new_island = set(self.island)
                new_right = set(self.right)
                
                if self.boat_location == 0:  
                    new_left.remove(p1)
                    new_island.add(p1)
                    new_boat = 1
                elif self.boat_location == 1:
                    new_island.remove(p1)
                    if i == 0:  
                        new_left.add(p1)
                        new_boat = 0
                    else:
                        new_right.add(p1)
                        new_boat = 2
                else:  
                    new_right.remove(p1)
                    new_island.add(p1)
                    new_boat = 1
                
                new_state = State(new_left, new_island, new_right, new_boat)
                if new_state.is_valid():
                    moves.append(new_state)
            
            for p2 in source:
                if p1 != p2:
                    for i, dest in enumerate(destinations):
                        new_left = set(self.left)
                        new_island = set(self.island)
                        new_right = set(self.right)
                        
                        if self.boat_location == 0:  
                            new_left.remove(p1)
                            new_left.remove(p2)
                            new_island.add(p1)
                            new_island.add(p2)
                            new_boat = 1
                        elif self.boat_location == 1:  
                            new_island.remove(p1)
                            new_island.remove(p2)
                            if i == 0:  # To Left
                                new_left.add(p1)
                                new_left.add(p2)
                                new_boat = 0
                            else:  
                                new_right.add(p1)
                                new_right.add(p2)
                                new_boat = 2
                        else:  
                            new_right.remove(p1)
                            new_right.remove(p2)
                            new_island.add(p1)
                            new_island.add(p2)
                            new_boat = 1
                        
                        new_state = State(new_left, new_island, new_right, new_boat)
                        if new_state.is_valid():
                            moves.append(new_state)
        
        return moves

husbands = list(range(1, n+1))
wives = [-h for h in husbands] 
people = husbands + wives

start_state = State(set(people), set(), set(), 0)

def print_solution(path):
    print(f"Solution found with {len(path)} steps:")
    for i, state in enumerate(path):
        print(f"\nStep {i}:")
        print(state)
    print("\nAll couples successfully crossed the river!")

def dfs(start_state):
    start_time = time.time()
    stack = [(start_state, [start_state])]
    visited = set([start_state])
    
    while stack:
        state, path = stack.pop()
        
        if state.is_goal():
            end_time = time.time()
            print(f"\nDFS took {end_time - start_time:.4f} seconds")
            return path
        
        for next_state in state.get_possible_moves():
            if next_state not in visited:
                visited.add(next_state)
                stack.append((next_state, path + [next_state]))
    
    return None

def backtracking(state, path, visited):
    if state.is_goal():
        return path
    
    for next_state in state.get_possible_moves():
        if next_state not in visited:
            visited.add(next_state)
            result = backtracking(next_state, path + [next_state], visited)
            if result:
                return result
            visited.remove(next_state)  
    
    return None

def solve_with_backtracking(start_state):
    start_time = time.time()
    visited = set([start_state])
    solution = backtracking(start_state, [start_state], visited)
    end_time = time.time()
    print(f"\nBacktracking took {end_time - start_time:.4f} seconds")
    return solution

def heuristic(state):
    return len(state.left) + len(state.island)

def a_star(start_state):
    start_time = time.time()
    counter = itertools.count()  # Unique sequence count for tiebreaking
    frontier = [(heuristic(start_state), 0, next(counter), start_state, [start_state])]
    visited = set([start_state])
    step_cost = 1
    
    while frontier:
        _, cost, _, state, path = heapq.heappop(frontier)
        
        if state.is_goal():
            end_time = time.time()
            print(f"\nA* took {end_time - start_time:.4f} seconds")
            return path
        
        for next_state in state.get_possible_moves():
            if next_state not in visited:
                visited.add(next_state)
                next_cost = cost + step_cost
                heapq.heappush(frontier, (
                    next_cost + heuristic(next_state), 
                    next_cost, 
                    next(counter), 
                    next_state, 
                    path + [next_state]
                ))
    
    return None

print("Starting state:")
print(start_state)
print("\nSolving with DFS:")
dfs_stime = time.time()
dfs_solution = dfs(start_state)
dfs_etime = time.time()
if dfs_solution:
    print_solution(dfs_solution)
else:
    print("No solution found with DFS.")

print("\nSolving with Backtracking:")
bt_stime = time.time()
bt_solution = solve_with_backtracking(start_state)
bt_etime = time.time()
print(f"Backtracking took {bt_etime - bt_stime:.4f} seconds")

if bt_solution:
    print_solution(bt_solution)
else:
    print("No solution found with Backtracking.")

print("\nSolving with A*:")
as_stime = time.time()
as_solution = a_star(start_state)
as_etime = time.time()
print(f"A* took {etime - stime:.4f} seconds")

if as_solution:
    print_solution(as_solution)
else:
    print("No solution found with A*.")

print("\nAll algorithms completed.")
#print all times in a table
print(f"{'Algorithm':<15}{'Time (s)':<15}")
print(f"{'DFS':<15}{dfs_etime - dfs_stime:<15.4f}")
print(f"{'Backtracking':<15}{bt_etime - bt_stime:<15.4f}")
print(f"{'A*':<15}{as_etime - as_stime:<15.4f}")

Starting state:
Left: [-2, -1, 1, 2]
Island: []
Right: []
Boat at: Left

Solving with DFS:

DFS took 0.0007 seconds
Solution found with 15 steps:

Step 0:
Left: [-2, -1, 1, 2]
Island: []
Right: []
Boat at: Left

Step 1:
Left: [1, 2]
Island: [-2, -1]
Right: []
Boat at: Island

Step 2:
Left: [-2, 1, 2]
Island: [-1]
Right: []
Boat at: Left

Step 3:
Left: [-2]
Island: [-1, 1, 2]
Right: []
Boat at: Island

Step 4:
Left: [-2, -1]
Island: [1, 2]
Right: []
Boat at: Left

Step 5:
Left: []
Island: [-2, -1, 1, 2]
Right: []
Boat at: Island

Step 6:
Left: []
Island: [1, 2]
Right: [-2, -1]
Boat at: Right

Step 7:
Left: []
Island: [-1, 1, 2]
Right: [-2]
Boat at: Island

Step 8:
Left: [-1, 1]
Island: [2]
Right: [-2]
Boat at: Left

Step 9:
Left: [-1]
Island: [1, 2]
Right: [-2]
Boat at: Island

Step 10:
Left: [-1]
Island: []
Right: [-2, 1, 2]
Boat at: Right

Step 11:
Left: [-1]
Island: [-2]
Right: [1, 2]
Boat at: Island

Step 12:
Left: [-2, -1]
Island: []
Right: [1, 2]
Boat at: Left

Step 13:
Left: []
I